# 1 Introduction
First, we have to activate the environment. The first run may take a while as external libraries will be downloaded and compiled.

In [1]:
Pkg.activate(".")

In [3]:
include("src/Challenge/init.jl")

evaluate (generic function with 1 method)

In [4]:
function generate_ansact_param(step_qc_exp, maxItr=1000)
    # Generate step qc
    step_qc = generate_empty_circuit()
    ansact = generate_ansact()
    # append
    append!(step_qc, step_qc_exp)
    append!(step_qc, ansact)
    length(getparameters(ansact))
    # Add measures
    addMeasuresOS(step_qc)

    # Set random parameters
    start_params = getRandParameters(step_qc)
    setparameters!(ansact, start_params)


    loss(params) = loss_expected_zero_state(execute(backend, step_qc, params))
    dloss(params) = real(loss'(params))
    of = OptimizationFunction(false, (x) -> (loss(x), dloss(x)), loss)

    val, xparams, itr = gradientDescent(of, start_params, α=0.01, maxItr=maxItr,
                                  argsArePeriodic=true, isExpectedZero=true, ϵ=1e-4, debug=false, useBigValInc=true)

    return xparams
end

generate_ansact_param (generic function with 2 methods)

In [5]:
trotter_steps = 10
trotter_steps_arr = [2, 2, 2, 2, 2]

5-element Vector{Int64}:
 2
 2
 2
 2
 2

In [6]:

errs = []
errsai = []
ts = [π/4, π/2, 3*π/4, π]
#ts = [π]
best_param = []
best_qc = nothing
for t in ts
    println("Start for t=$t")
    qc, params, params2, params3 = generate_circuit(trotter_steps, trotter_steps, t)

    full_qc = generate_empty_circuit()
    full_qc.x([3, 5])
    append!(full_qc, qc)
    addMeasuresOS(full_qc)

    # caclulate the end state
    uerr = check_simulation_err(full_qc, t)

    ###########################################################################
    do_iter = 0
    do_check = true
    first_step = true
    inv_ansact = []
    for st in trotter_steps_arr
        step_qc_exp, _, _, _ = generate_circuit(trotter_steps, st, t, params, params2, params3, init=first_step)
        #step_qc_expmat = tomatrix(step_qc)
        do_iter += st
        println("Start AI TS $do_iter")

        if first_step
            opt_step_qc_exp = step_qc_exp
        else
            opt_step_qc_exp = generate_empty_circuit()
            append!(opt_step_qc_exp, inv_ansact)
            append!(opt_step_qc_exp, step_qc_exp)
        end

        # find best parameters
        best_params = generate_ansact_param(opt_step_qc_exp, 10000)

        # Generate ansact
        ansact = generate_ansact()
        setparameters!(ansact, best_params)
        inv_ansact = inv(ansact)
        bindparameters!(inv_ansact)

        # Check step reaults
        if do_check && do_iter < trotter_steps
            check_qc, _, _, _ = generate_circuit(trotter_steps, trotter_steps-do_iter, t, params, params2, params3)

            # Simulate full circuit
            check_step_qc_full = generate_empty_circuit()
            append!(check_step_qc_full, inv_ansact)
            append!(check_step_qc_full, check_qc)
            # Add measures
            addMeasuresOS(check_step_qc_full)

            # do check
            check_err = check_simulation_err(check_step_qc_full, t)
            check_err2 = check_circuits_err(full_qc, check_step_qc_full)
            println("Check value: $check_err, $check_err2")
            @assert check_err2 < 10e-2
        end

        # next initmat
        first_step = false
    end

    ###########################################################################
    addMeasuresOS(inv_ansact)
    check_err = check_simulation_err(inv_ansact, t)
    check_err2 = check_circuits_err(full_qc, inv_ansact)
    append!(errs, check_err)
    append!(errsai, check_err2)
end

Start for t=0.7853981633974483
Start AI TS 2
Check value: 0.05173132946987757, 0.05374363435608954
Start AI TS 4
Check value: 0.053534419845862144, 0.05554672473207412
Start AI TS 6
Check value: 0.05546832249340871, 0.05748062737962068
Start AI TS 8
Check value: 0.05206070091787305, 0.054073005804085035
Start AI TS 10
Start for t=1.5707963267948966
Start AI TS 2
Check value: 0.0005816270105747522, 0.00036220769992775044
Start AI TS 4
Check value: 0.003928152533207231, 0.003986767412904203
Start AI TS 6
Check value: 0.005965120621054119, 0.0059232146203066025
Start AI TS 8
Check value: 0.005083339768004516, 0.005141954647701488
Start AI TS 10


LoadError: InterruptException:

In [ ]:
println("==============================")
#println("The error is $(sum(errs)).")
println("The error Ai is $(mean(errsai)).")
println("==============================")